In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import anndata as ad

from shapely import force_3d

import matplotlib.pyplot as plt
import seaborn as sns
from spida.pl import plot_categorical, plot_continuous
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (4, 4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['axes.facecolor'] = 'white'

from datetime import datetime
curr_datetime = datetime.now().strftime("%Y%m%d")

In [ ]:
uci5224_coords = {
    "CAB" : (2, 2, 61.45),
    "CAH" : (-5, 0, 52.5),
    "CAT" : (10, -15, 87.6),
    "PU" : (12, -8, 59.0),
    "NAC": (0, -18, 52.5),
    "GP": (4, -10, 62.45),
    "SUBTH": (-5, -22, 73.8),
    "MGM1": (-10, -30, 80.0),
}

In [ ]:
adata = ad.read_h5ad('/home/x-aklein2/projects/aklein/BICAN/BG/data/BICAN_BG_PFV8_annotated_v2.h5ad', backed='r')
adata_uci5224 = adata[adata.obs['donor'] == "UCI5224"].to_memory()
adata_uci5224

### Orient

In [ ]:
from spida._constants import ren_to_exp_map

In [ ]:
df_adj = []
for _dsid in adata_uci5224.obs['dataset_id'].unique():
    _reg, _donor, _lab = _dsid.split("_")
    print(_dsid, _reg, _lab)
    
    adata_reg = adata_uci5224[adata_uci5224.obs['dataset_id'] == _dsid].copy()
    extent = adata_reg.obsm['spatial']

    gdf = gpd.GeoDataFrame(extent, columns=['x', 'y'], index=adata_reg.obs_names)
    gdf['geom'] = gpd.points_from_xy(gdf.x, gdf.y)
    gdf = gdf.set_geometry("geom")
    
    minx, miny, maxx, maxy = gdf.total_bounds
    gdf.geometry = gdf.geometry.translate(-minx, -miny)

    x_scale = maxx - minx
    y_scale = maxy - miny
    print(x_scale, y_scale)

    if _lab == "ucsd":
        _reg = ren_to_exp_map[_reg]

    x_y_translate = uci5224_coords[_reg][0]*1000, uci5224_coords[_reg][1]*1000 # mm to um
    z_val = uci5224_coords[_reg][2]*1000 # mm to um

    gdf.geometry = gdf.geometry.translate(x_y_translate[0], x_y_translate[1])
    gdf.geometry = gdf.geometry.apply(lambda x: force_3d(x, z=z_val))
    gdf['z'] = z_val
    gdf['x'] = gdf.geometry.x
    gdf['y'] = gdf.geometry.y
    df_adj.append(gdf[['x', 'y', 'z']])

    # adata_reg.obsm['spatial_donor'] = gdf[['x', 'y', 'z']].to_numpy()    
    # break
    # adata_uci5224.obs.loc[adata_uci5224.obs['dataset_id'] == _dsid, 'spatial_x'] = adata_uci5224.obs.loc[adata_uci5224.obs['dataset_id'] == _dsid, 'spatial_x'] - uci5224_coords[_dsid][0]
    # adata_uci5224.obs.loc[adata_uci5224.obs['dataset_id'] == _dsid, 'spatial_y'] = adata_uci5224.obs.loc[adata_uci5224.obs['dataset_id'] == _dsid, 'spatial_y'] - uci5224_coords[_dsid][1]
    # adata_uci5224.obs.loc[adata_uci5224.obs['dataset_id'] == _dsid, 'spatial_z'] = adata_uci5224.obs.loc[adata_uci5224.obs['dataset_id'] == _dsid, 'spatial_z'] - uci5224_coords[_dsid][2]
df_adj = pd.concat(df_adj)
adata_uci5224.obsm['spatial_donor'] = df_adj.loc[adata_uci5224.obs_names].to_numpy()

## Plot

In [ ]:
order_5224 = ["NAC", "CAH", "CAB", "PU", "GP", "SUBTH", "MGM1", "CAT"]
order_2424 = ["CAH", "NAC", "PU", "CAB", "GP", "SUBTH", "MGM1", "CAT"]
order_4723 = ["CAH", "NAC", "PU", "CAB", "GP", "SUBTH", "MGM1", "CAT"]
order_7648 = ["CAH", "PU", "NAC", "CAB", "GP", "SUBTH", "MGM1"]

In [ ]:
for _lab in adata_uci5224.obs['replicate'].unique(): 
    fig, axes = plt.subplots(1, len(order_5224), figsize=(5*len(order_5224), 4), dpi=300)
    for _i, _regs in enumerate(order_5224):
        adata_regs = adata_uci5224[(adata_uci5224.obs['replicate'] == _lab) & (adata_uci5224.obs['brain_region'] == _regs)].copy()
        plot_categorical(adata_regs, coord_base='spatial', cluster_col='RNA.Subclass', show=False, ax=axes[_i], show_legend=False)
        axes[_i].set_title(_regs)
    plt.suptitle(f"UCI5224")
    plt.show()

In [ ]:
adata_uci2424 = adata[adata.obs['donor'] == "UCI2424"].to_memory()
adata_uci2424

for _lab in adata_uci2424.obs['replicate'].unique(): 
    fig, axes = plt.subplots(1, len(order_2424), figsize=(5*len(order_2424), 4), dpi=300)
    for _i, _regs in enumerate(order_2424):
        adata_regs = adata_uci2424[(adata_uci2424.obs['replicate'] == _lab) & (adata_uci2424.obs['brain_region'] == _regs)].copy()
        plot_categorical(adata_regs, coord_base='spatial', cluster_col='RNA.Subclass', show=False, ax=axes[_i], show_legend=False)
        axes[_i].set_title(_regs)
    plt.suptitle(f"UCI2424")
    plt.show()

In [ ]:
adata_uci4723 = adata[adata.obs['donor'] == "UCI4723"].to_memory()
adata_uci4723

for _lab in adata_uci5224.obs['replicate'].unique(): 
    fig, axes = plt.subplots(1, len(order_4723), figsize=(5*len(order_4723), 4), dpi=300)
    for _i, _regs in enumerate(order_4723):
        adata_regs = adata_uci4723[(adata_uci4723.obs['replicate'] == _lab) & (adata_uci4723.obs['brain_region'] == _regs)].copy()
        plot_categorical(adata_regs, coord_base='spatial', cluster_col='RNA.Subclass', show=False, ax=axes[_i], show_legend=False)
        axes[_i].set_title(_regs)
    plt.suptitle(f"UCI4723")
    plt.show()

In [ ]:
adata_uwa7648 = adata[adata.obs['donor'] == "UWA7648"].to_memory()
adata_uwa7648

for _lab in adata_uwa7648.obs['replicate'].unique(): 
    fig, axes = plt.subplots(1, len(order_7648), figsize=(5*len(order_7648), 4), dpi=300)
    for _i, _regs in enumerate(order_7648):
        adata_regs = adata_uwa7648[(adata_uwa7648.obs['replicate'] == _lab) & (adata_uwa7648.obs['brain_region'] == _regs)].copy()
        plot_categorical(adata_regs, coord_base='spatial', cluster_col='RNA.Subclass', show=False, ax=axes[_i], show_legend=False)
        axes[_i].set_title(_regs)
    plt.suptitle(f"UWA7648")
    plt.show()

### Orient Plots

In [ ]:
for _dsid in adata_uci5224.obs['dataset_id'].unique():
    _reg, _donor, _lab = _dsid.split("_")
    print(_dsid, _reg, _lab)
    
    adata_reg = adata_uci5224[adata_uci5224.obs['dataset_id'] == _dsid].copy()

In [ ]:
plot_categorical(adata_uci5224, coord_base='spatial_donor', cluster_col='RNA.Subclass')

In [ ]:
adata_uci5224.uns['donor_orient'] = uci5224_coords

In [ ]:
# Load in a location config file (with the x, y, and z coords. of the given experiments)
# (x - anterior-posterior)
# (y - dorso-ventral)
# (z - rostro-caudal / medio-lateral...)
# By donor x brain_region